# Importando bibliotecas e data set

Baixando bibliotecas

## Importando bibliotecas

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from datetime import datetime
from joblib import dump

## Importando datasets

In [ ]:
data_2020_2021 = pd.read_csv("../dados/base_inteli 2020_2021.csv", sep=";" , encoding='utf8')
df_1 = pd.DataFrame(data_2020_2021)

In [ ]:
data_2022_2023 = pd.read_csv("../dados/base_inteli_2022_2023.csv", sep =";", encoding='utf8')
df_2 = pd.DataFrame(data_2022_2023)

## Aplicação do dolar e da taxa SELIC no dataset

In [ ]:
dolar = pd.read_csv("../dados/dolar.csv", sep =";", encoding='utf8')
df_dolar = pd.DataFrame(dolar)

># convertendo "data" do dataset do dólar de string para "date", de forma a facilitar o pré-processamento


In [ ]:
df_dolar['data'] = pd.to_datetime(df_dolar['Data e hora da Cotacao']).dt.date

In [ ]:
df_dolar = df_dolar.drop(columns=['Data e hora da Cotacao'], axis=1)

In [ ]:
df_dolar

In [ ]:
#selecionando o intervalo do dataset da Mobly e transformando para realizar o merge entre os dataframes
date_threshold = datetime.strptime('2019-05-01', '%Y-%m-%d').date()
df_dolar = df_dolar[df_dolar['data'] > date_threshold]

In [ ]:
#importando dataset da taxa selic
data_selic = pd.read_csv('../dados/selic.csv', sep = ";", encoding='utf8')
df_selic = pd.DataFrame(data_selic)

In [ ]:
# convertendo data do dataset da selic de string para date, de forma a facilitar o pré-processamento
df_selic['data'] = df_selic['data'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').date())

In [ ]:
df_selic['data'] = pd.to_datetime(df_selic['data'])

In [ ]:
df_selic['valor'] = df_selic['valor'].str.replace(',', '.').astype(float)

In [ ]:
df_selic['mes'] = df_selic['data'].dt.month

In [ ]:
df_selic['ano'] = df_selic['data'].dt.year

In [ ]:
df_media = df_selic.groupby([df_selic['data'].dt.year, df_selic['data'].dt.month])['valor'].mean()

In [ ]:
df_media = df_media.rename_axis(['ano','mes'])

In [ ]:
df_selic = pd.merge(df_selic, df_media, how='left', left_on=['mes','ano'], right_on=['mes', 'ano'])

In [ ]:
df_selic = df_selic.drop(columns=['mes', 'ano', 'valor_x'], axis=1)

In [ ]:
df_selic.rename(columns= {'valor_y': 'selic'}, inplace = True)

In [ ]:
#selecionando o intervalo do dataset da Mobly e transformando para realizar o merge entre os dataframes
date_threshold = datetime.strptime('2019-12-30', '%Y-%m-%d')
df_selic = df_selic[df_selic['data'] > date_threshold]

In [ ]:
df_selic.head()

## Concatenando o data set em um dataframe

In [ ]:
df = pd.concat([df_1,df_2])

In [ ]:
#convertendo o formato da data para realizar o merge
df['date'] = pd.to_datetime(df['date'])
df_dolar['data'] = pd.to_datetime(df_dolar['data'])

In [ ]:
df = df.merge(df_dolar, how='left', left_on='date', right_on='data')

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['data'], axis=1)

In [ ]:
#convertendo o formato da data para realizar o merge
df_selic['data'] = pd.to_datetime(df_selic['data'])

In [ ]:
df = pd.merge(df, df_selic, how='left', left_on='date', right_on='data')

In [ ]:
num_nan = df['selic'].isnull().sum()

In [ ]:
num_nan = df['Cotacao Compra'].isnull().sum()

In [ ]:
#preenchendo dados vazios com valores mais próximos
df['Cotacao Compra'].fillna(method='bfill', inplace=True)

In [ ]:
df = df.drop(columns=['Cotacao Venda'], axis=1)

In [ ]:
df.rename(columns={'Cotacao Compra': 'dolar'}, inplace=True)

In [ ]:
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)

In [ ]:
#preenchendo dados vazios com valores mais próximos
df['selic'].fillna(method='ffill', inplace=True)
df['selic'].fillna(method='bfill', inplace=True)

In [ ]:
df = df.drop(columns=['data'], axis=1)

In [ ]:
df['selic'] = pd.to_numeric(df['selic'], errors='coerce')

In [ ]:
df.columns

# Pré-Processamento

## Detecção de dados duplicados

In [ ]:
duplicados = df[df.duplicated(keep='first')]

## Eliminação de colunas desnecessárias





> Eliminamos a coluna mobly_item pois não interfere no nosso modelo.



In [ ]:
df = df.drop(columns=['mobly_item'])

Eliminamos a coluna weekday_name pois tratamos a data popsteriormente.

In [ ]:
df = df.drop(columns=['weekday_name'])

> Eliminamos as colunas relacionadas ao "bundle" devido a colinearidade com a coluna "items_sold".

In [ ]:
df = df.drop(columns=['flag_bundle'])

In [ ]:
df = df.drop(columns=['revenue_bundle'])

In [ ]:
df = df.drop(columns=['items_sold_bundle'])

> Eliminamos a coluna "supplier_delivery_time" pois não interfere no nosso modelo

In [ ]:
df = df.drop(columns=["supplier_delivery_time"])

> Eliminamos as colunas relacionadas às medidas do SKU pois não interfere no nosso modelo

In [ ]:
df = df.drop(columns=["sku_height"])

In [ ]:
df = df.drop(columns=["sku_width"])

In [ ]:
df = df.drop(columns=["sku_weight"])

In [ ]:
df = df.drop(columns=["sku_length"])

> Código para verificar o número de células vazias

In [ ]:
df.isnull().sum()

## Alterando colunas

Transformando coluna "stock_qty" para 1 (tem no estoque) ou 0 (não tem no estoque).

In [ ]:
df.loc[df['stock_qty'] >= 1, 'stock_qty'] = 1

Criando duas novas colunas no DataFrame: 'is_national', que indica se o produto é de origem Nacional, e 'can_provide_material', que indica se a empresa pode fornecer a matéria-prima para produção com base nas condições fornecidas.

In [ ]:
df['is_national'] = df['origin_country'].map({'Nacional': True, 'Importado': False})

df['can_provide_material'] = (df['is_national'] & (df['process_costing'] == 'yes'))

df = df.drop(columns=['process_costing'])
df = df.drop(columns=['origin_country'])

df.tail(10)

## Preenchimento das colunas com lagging

Com base nas orientações dos parceiros sobre o funcionamento da API de comparação dos preços no Google Shopping, foi percebido que há um preenchimento semanal por SKU, tal que os outros dias da semana permanece o mesmo valor, mas ele não é registrado. Assim, as linhas "sem registro" foram preenchidas com lagging a partir do valor mais próximo da data por SKU.

In [ ]:
df = df.sort_values(by=['sku', 'date'])

In [ ]:
#preenchendo as linhas sem resgitro de winning_price com o preço da semana, preenchendo linhas nulas acima e abaixo
df['winning_price'] = df.groupby('sku')['winning_price'].ffill()
df['winning_price'] = df.groupby('sku')['winning_price'].bfill()

In [ ]:
# Preenchendo as linhas vazias do "winning_price" pois são itens únicos na MObly e não possuem concorrentes 
df['winning_price'].fillna(df['unit_price'], inplace=True)

In [ ]:
#susbtituindo "Sem |Registro" por NAN, para aplicar os métodos ffill e bfill
df['price_status'].replace('Sem Registro', np.nan, inplace=True)

In [ ]:
#preenchendo as linhas sem resgitro de price_status com o preço da semana, preenchendo linhas nulas acima e abaixo
df['price_status'] = df.groupby('sku')['price_status'].ffill()
df['price_status'] = df.groupby('sku')['price_status'].bfill()

In [ ]:
#preenchendo as linhas nulas com "Uníco Disponpivel", pois não possuem um status no Google Shopping, por serem itens únicos da Mobly
df['price_status'].replace(np.nan, 'Único Disponível', inplace=True)

## Codificação das variáveis categóricas

Verificação dos tipos das variáveis para identificar as categóricas que restaram após os tratamentos de colunas desnecessárias, dados faltantes e inconsistentes para serem codificadas.

In [ ]:
df.dtypes

Ferramenta para transformar variáveis categóricas não numéricas em uma representação numérica.

In [ ]:
le = LabelEncoder()

Codificação de variáveis categóricas com o LabelEncoder para variáveis que possuem mais de 6 valores únicos e armazenamento do dicionário com o mapeamento para decodificação posterior no notebook do modelo.

In [ ]:
columns_to_encode = ['sku', 'anchor_category', 'product_department', 'product_category', 'sku_color']

In [ ]:
decoding_dicts = {}

for column in columns_to_encode:
    df[column] = le.fit_transform(df[column])
    decoding_dicts[column] = dict(zip(range(len(le.classes_)), le.classes_))

In [ ]:
dump(decoding_dicts, 'decoding_dicts.joblib')

Codificação de variáveis categóricas com o get dummies do pandas (técnica do OneHotEncoder) para variáveis que possuem até 5 valores únicos

In [ ]:
df = pd.get_dummies(df, columns=['shipment_type', 'price_status'], drop_first='True')

## Tratamento data

Tratamento da váriavel date: Separação em outras colunas

In [ ]:
# Extraindo os componentes da data
df['ano'] = df['date'].dt.year
df['mes'] = df['date'].dt.month
df['dia'] = df['date'].dt.day

# 0: Segunda-feira, 1: Terça-feira, etc.
df['semana_do_ano'] = df['date'].dt.isocalendar().week
df['trimestre'] = df['date'].dt.quarter
df['dia_do_ano'] = df['date'].dt.dayofyear
df['dia_da_semana'] = df['date'].dt.weekday

# Retorna True para sábado e domingo
df['fim_de_semana'] = df['date'].dt.weekday >= 5  

# Transformação cíclica para o dia do ano
df['dia_do_ano_sin'] = np.sin(2 * np.pi * df['dia_do_ano']/365)
df['dia_do_ano_cos'] = np.cos(2 * np.pi * df['dia_do_ano']/365)

# Transformação cíclica para o dia da semana
df['dia_da_semana_sin'] = np.sin(2 * np.pi * df['dia_da_semana']/7)
df['dia_da_semana_cos'] = np.cos(2 * np.pi * df['dia_da_semana']/7)

Tratando a coluna `date` para tirar segundos, horas ...

In [ ]:
df.date = pd.to_datetime(df.date).dt.date

Agora a visualização das últimas 10 linhas do dataframe

In [ ]:
df.tail(10)

## Correção dummies

Tranformação das variáveis codificadas com dummies "True" e "False" em 1 e 0

In [ ]:
dummies_dict = {
    True: 1,
    False: 0
}

df = df.replace(dummies_dict)

> Eliminando dados anteriores a primeira ocorrência de cada SKU

In [ ]:
def achar_primeira_ocorrencia(df):
    condicao = (df.items_sold > 0) | (df.avg_website_visits_last_week > 0)
    if condicao.any():
        return condicao.idxmax()
    else:
        return None
    

mask = df.groupby("sku").apply(achar_primeira_ocorrencia).reset_index(name="achar_primeira_ocorrencia")
df = df.merge(mask, on="sku", how='left')
manter_linhas = df.index >= df["achar_primeira_ocorrencia"]

#Filtragem do dataframe usando a máscara

df_filtrado = pd.DataFrame(df[manter_linhas].drop(columns=['achar_primeira_ocorrencia']))


# Armazenamento df

In [ ]:
df_1 = df

%store df_1